<a href="https://colab.research.google.com/github/Tiger-A/GoogleCollab/blob/main/%D0%92%D0%B5%D0%B1%D0%B8%D0%BD%D0%B0%D1%80_25_%D0%BC%D0%B0%D1%8F_%D0%9D%D0%B5%D1%80%D0%B9%D0%BE_%D0%BF%D1%80%D0%BE%D0%B4%D0%B0%D0%B6%D0%BD%D0%B8%D0%BA_2_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загружаем функции


In [ ]:
!pip -q install --upgrade tiktoken
!pip -q install langchain openai chromadb
!pip -q install gspread oauth2client

from langchain.llms import OpenAI
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re
import logging
logging.getLogger("langchain.text_splitter").setLevel(logging.ERROR) # игнорирование предупреждений
logging.getLogger("chromadb").setLevel(logging.ERROR)
import os
import openai
import tiktoken
import re


class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def set_key():
    password_input = widgets.Password(
        description='Введите пароль:', 
        layout=widgets.Layout(width='500px'),
        style={'description_width': 'initial', 'white-space': 'pre-wrap', 'overflow': 'auto'})
    login_button = widgets.Button(description='Авторизация')
    output = widgets.Output()

    def on_button_clicked(_):
        with output:
            openai.api_key = password_input.value
            os.environ["OPENAI_API_KEY"] = openai.api_key
            print(f'{bcolors.OKGREEN}{bcolors.BOLD}Ключ сохранен!{bcolors.ENDC}')
            password_input.layout.display = 'none'
            login_button.layout.display = 'none'

    login_button.on_click(on_button_clicked)
    display(widgets.VBox([password_input, login_button, output]))

def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

def create_search_index(text: str) -> Chroma:
    return create_embedding(text)


def load_prompt(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return f'{text}'


def create_embedding(data):
    def num_tokens_from_string(string: str, encoding_name: str) -> int:
        """Returns the number of tokens in a text string."""
        encoding = tiktoken.get_encoding(encoding_name)
        num_tokens = len(encoding.encode(string))
        return num_tokens

    source_chunks = []
    splitter = CharacterTextSplitter(separator="\n", chunk_size=1024, chunk_overlap=0)

    for chunk in splitter.split_text(data):
        source_chunks.append(Document(page_content=chunk, metadata={}))

    # Создание индексов документа
    search_index = Chroma.from_documents(source_chunks, OpenAIEmbeddings(), )

    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
    #print('\n ===========================================: ')
    #print('Количество токенов в документе :', count_token)
    #print('ЦЕНА запроса:', 0.0004*(count_token/1000), ' $')
    return search_index

def answer(system, topic, temp = 1):    
    messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": topic}
      ]

    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=temp
      )

    return completion.choices[0].message.content

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """
    Возвращает количество токенов, используемых списком сообщений.
    """
    try:
        # Пытаемся получить кодировку для выбранной модели
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        # Если кодировка для выбранной модели не найдена, используем кодировку "cl100k_base"
        encoding = tiktoken.get_encoding("cl100k_base")
    # Если выбранная модель это "gpt-3.5-turbo-0301"
    if model == "gpt-3.5-turbo-0301": 
        # Инициализируем счетчик токенов
        num_tokens = 0
        # Проходимся по каждому сообщению в списке сообщений
        for message in messages:
            # Каждое сообщение обрамляется токенами <im_start> и <im_end>, а также символами новой строки, всего 4 токена
            num_tokens += 4 
            # Проходимся по каждому полю в сообщении (ключ и значение)
            for key, value in message.items():
                # Считаем количество токенов в значении и добавляем их в счетчик токенов
                num_tokens += len(encoding.encode(value))
                # Если ключ это "name", то это означает что роль (role) опущена
                if key == "name":  
                    # Роль всегда требуется и всегда занимает 1 токен, так что вычитаем 1 из счетчика
                    num_tokens += -1  
        # Каждый ответ начинается с токена <im_start>assistant, так что добавляем 2 в счетчик
        num_tokens += 2  
        # Возвращаем количество токенов
        return num_tokens
    else:
        # Если выбранная модель не поддерживается, генерируем исключение
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")


def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)


def dialog():
    user = ''
    dialog = ''

    print(f'{bcolors.OKBLUE}{bcolors.BOLD}С чем связан ваш интерес к искусственному интеллекту?{bcolors.ENDC}')

    while user.lower() not in ['stop', 'exit', 'выход']:
        user = input('Клиент: ')
        if user == 'Stop': break

        dialog += '\n\n' + 'Клиент: ' + user
        add_dialog = answer(expert_prompt, user)
        
        dialog += '\n\n' + 'Менеджер: ' + add_dialog      
        print(f'\n{bcolors.OKBLUE}{bcolors.BOLD}Менеджер:{bcolors.ENDC} {insert_newlines(add_dialog)}')
        report = answer(validation_prompt, dialog)
        answer_text = answer(action_prompt, report)

        print(f'\n{bcolors.OKGREEN}{bcolors.BOLD}Отчёт системы:\n {bcolors.ENDC}{report}')
        print(f'\n{bcolors.HEADER}{bcolors.BOLD}Менеджер: {bcolors.ENDC}{insert_newlines(answer_text)}\n\n')

    return dialog

def answer_index(system, topic, search_index, temp=1, verbose=0):
    
    # Selecting documents similar to the question 
    docs = search_index.similarity_search(topic, k=5)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)

    messages = [
        {"role": "system", "content": system + f"{message_content}"},
        {"role": "user", "content": topic}
    ]

    if verbose: print('\n ===========================================: ')
    if verbose: print(f"{num_tokens_from_messages(messages, 'gpt-3.5-turbo-0301')} tokens used for the question")

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temp
    )

    if verbose: print('\n ===========================================: ')
    if verbose: print(f'{completion["usage"]["total_tokens"]} total tokens used (question-answer).')
    if verbose: print('\n ===========================================: ')
    answer = insert_newlines(completion.choices[0].message.content)
    #print('ANSWER : \n', answer)
    return answer  # возвращает ответ 

def get_chatgpt_ansver3(system, topic, search_index, temp = 1):
    
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": topic}
    ]

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temp
    )
    print('Ответ менеджера : \n', insert_newlines(completion.choices[0].message.content))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.5/873.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.6/922.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
def num_tokens_from_history(messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            question, answer = message  # распаковываем кортеж
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            num_tokens += len(encoding.encode(question))
            num_tokens += len(encoding.encode(answer))
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.""")

In [ ]:
def summarize_questions(dialog):
    # Применяем модель GPT-3 для суммаризации вопросов
    messages = [
        {"role": "system", "content": "Ты - ассистент отдела продаж, основанный на AI. Ты умеешь профессионально суммаризировать присланные тебе диалоги менеджера и клиента. Твоя задача - суммаризировать диалог, который тебе пришел."},
        {"role": "user", "content": "Суммаризируй следующий диалог менеджера по продажам и клиента: " + " ".join(dialog)}
    ]
    
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.3,  # Используем более низкую температуру для более определенной суммаризации
        max_tokens=500  # Ограничиваем количество токенов для суммаризации
    )
    
    return completion.choices[0].message.content

In [ ]:
def answer_user_question(system_doc_url: str, knowledge_base_url: str, user_question: str) -> str:
    global question_history
    system_doc_text = load_document_text(system_doc_url)
    knowledge_base_text = load_document_text(knowledge_base_url)

    # Создаем индексы поиска
    knowledge_base_index = create_search_index(knowledge_base_text)

    # Если в истории более одного вопроса, применяем суммаризацию
    summarized_history = ""
    if len(question_history) > 0:
        summarized_history = "Вот краткий обзор предыдущего диалога: " + summarize_questions([q + ' ' + (a if a is not None else '') for q, a in question_history])

    # Добавляем явное разделение между историей диалога и текущим вопросом
    input_text =summarized_history + "\n\nТекущий вопрос: " + user_question

    # Извлечение наиболее похожих отрезков текста из базы знаний и получение ответа модели
    answer_text = answer_index(system_doc_text, input_text, knowledge_base_index, temp=temperature, verbose=verbose)
    
    # Добавляем вопрос пользователя и ответ системы в историю
    question_history.append((user_question, answer_text if answer_text is not None else ''))

    # Выводим суммаризированный текст, который видит модель
    if summarized_history != "":
        print("Вот суммаризированный текст, который видит модель:\n", summarized_history)
    
    return answer_text


In [ ]:
def answer_user_question_dialog(system_doc_url: str, knowledge_base_url: str, user_question: str, question_history: list) -> str:
    
    system_doc_text = load_document_text(system_doc_url)
    knowledge_base_text = load_document_text(knowledge_base_url)

    # Создаем индексы поиска
    knowledge_base_index = create_search_index(knowledge_base_text)

    # Если в истории более одного вопроса, применяем суммаризацию
    summarized_history = ""
    if len(question_history) > 0:
        summarized_history = "Вот краткий обзор предыдущего диалога: " + summarize_questions([q + ' ' + (a if a is not None else '') for q, a in question_history])

    # Добавляем явное разделение между историей диалога и текущим вопросом
    input_text =summarized_history + "\n\nТекущий вопрос: " + user_question

    # Извлечение наиболее похожих отрезков текста из базы знаний и получение ответа модели
    answer_text = answer_index(system_doc_text, input_text, knowledge_base_index, temp=temperature, verbose=verbose)
    
    # Добавляем вопрос пользователя и ответ системы в историю
    question_history.append((user_question, answer_text if answer_text is not None else ''))

    # Выводим суммаризированный текст, который видит модель
    if summarized_history != "":
        print(summarized_history)
    
    return answer_text


In [ ]:
def run_dialog(system_doc_url, knowledge_base_url):
    question_history = []
    dialog = ""
    while True:
        user_question = input('Клиент: ')
        if user_question.lower() == 'stop':
            break
        answer = answer_user_question_dialog(system_doc_url, knowledge_base_url, user_question, question_history)
        dialog += f'\nКлиент: {user_question} \n Менеджер: {answer}'
        print('\nМенеджер: ', answer)

    return 

# Вводим ключ и запускаем диалог

In [ ]:
temperature=1
verbose=0
question_history = []

In [ ]:
knowledge_base_url = 'https://docs.google.com/document/d/1P87cjAIJ3etAteDKjpyZycxXlFaC1EcgLE9UxhrIlMY' # база знаний

In [ ]:
system_doc_url = 'https://docs.google.com/document/d/1vT5CNdoryrcP6Keqf5y7jauSj0Ru797xvpjTu9pZ0wo' # инструкция

In [ ]:
set_key()

In [ ]:
run_dialog(system_doc_url, knowledge_base_url)

Клиент: Привет. Меня зовут Света. Смотрела ваши реалити. Не совсем понятно, смогу ли я

Менеджер:   Здравствуйте, Света! Расскажите, пожалуйста, для чего вы хотели бы изучать AI? Это поможет мне подобрать наиболее подходящий вариант обучения для вас. Кроме того, наша
 компания предлагает обучение с нуля, поэтому даже если вы не имеете предыдущего опыта в этой области, мы сможем помочь вам стать AI-разработчиком. Если вы готовы начать
 обучение, предлагаю вам сейчас зарегистрироваться на нашем курсе. Наши опытные преподаватели помогут вам узнать и овладеть технологиями AI уже через несколько дней после
 начала обучения.
Клиент: ой, нет, думаю, у меня не получится. Я никогда не программировала, и математику не помню совсем, я ее только в школе проходила 100 лет назад
Вот краткий обзор предыдущего диалога: Клиент по имени Света интересуется изучением AI, но не уверена, сможет ли она это сделать. Менеджер предлагает ей обучение с нуля и помощь в становлении AI-разработчиком. Он также предл